In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import sqlite3

In [3]:
# Парсинг страницы
l_df = []
n = 0

url = "https://hh.ru/search/vacancy?no_magic=true&L_save_area=true&\
text=middle+Python+developer&search_field=name&excluded_text=&salary=&\
currency_code=RUR&experience=doesNotMatter&order_by=relevance&search_period=0&\
items_on_page=100&page={}".format(n)
user_agent = {'User-agent': 'Mozilla/5.0'}
while len(l_df)<=100:
    result = requests.get(url, headers=user_agent)
    if result.status_code == 200:
        soup = BeautifulSoup(result.content.decode(), 'lxml')
        l = []
        for title in soup.select('[class="serp-item__title"]'):
            l.append(title.attrs['href'])
        while len(l_df)<=100:
            for href in l:
                url = href
                user_agent = {'User-agent': 'Mozilla/5.0'}
                result = requests.get(url, headers=user_agent)
                if result.status_code == 200:
                    print(href)
                    soup = BeautifulSoup(result.content.decode(), 'lxml')
                    if soup.find('div', attrs={'class': 'vacancy-title vacancy-title_archive'}):
                        print(i,'- в Архиве!')
                    else:    
                        company_name = soup.find('span', attrs={'class': 'vacancy-company-name'}).text
                        position = soup.find('h1', attrs={'data-qa': 'vacancy-title'}).text
                        job_description = soup.find('div', attrs={'data-qa': 'vacancy-description'}).text
                        skills = []
                        for j in soup.select('[data-qa="bloko-tag__text"]'):
                            skills.append(j.text)
                        key_skills = ','.join(skills)
                        # key_skills = str(skills).strip('[]')
                        # key_skills = soup.select('[data-qa="bloko-tag__text"]')
                        ls = [company_name, position, job_description, key_skills]
                        l_df.append(ls)
                        time.sleep(1)
                else:
                    print('Ошибка обращения к серверу. Код ответа: ', result.status_code) 
        n += 1            

https://kazan.hh.ru/vacancy/82841724?from=vacancy_search_list&query=middle+Python+developer
https://kazan.hh.ru/vacancy/83300859?from=vacancy_search_list&query=middle+Python+developer
https://kazan.hh.ru/vacancy/83458412?from=vacancy_search_list&query=middle+Python+developer
https://kazan.hh.ru/vacancy/83438237?from=vacancy_search_list&query=middle+Python+developer
https://kazan.hh.ru/vacancy/82825127?from=vacancy_search_list&query=middle+Python+developer
https://kazan.hh.ru/vacancy/82858718?from=vacancy_search_list&query=middle+Python+developer
https://kazan.hh.ru/vacancy/83409596?from=vacancy_search_list&query=middle+Python+developer
https://kazan.hh.ru/vacancy/83460735?from=vacancy_search_list&query=middle+Python+developer
https://kazan.hh.ru/vacancy/83409394?from=vacancy_search_list&query=middle+Python+developer
https://kazan.hh.ru/vacancy/82119961?from=vacancy_search_list&query=middle+Python+developer
https://kazan.hh.ru/vacancy/83279670?from=vacancy_search_list&query=middle+Pytho

In [ ]:
# Получение данных из api
l_df = []
n = 0

# text.field=name&resume_search_logic=all&
# url = "https://api.hh.ru/vacancies?text=middle python developer&per_page=10&page={}".format(n)
url = "https://api.hh.ru/vacancies"
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
params = {'text': 'middle Python developer', 
          'per_page': 20, 
          'page': n,
          'text.field': 'name',
          'resume_search_logic': 'all'
          }
# user_agent = {'User-agent': 'Mozilla/5.0'}
while len(l_df)<=100:
    result = requests.get(url, headers=headers, params=params)
    if result.status_code == 200:
        vacancies = result.json().get('items')
        l = []
        for vacancy in vacancies:
            l.append(vacancy['url'])
        while len(l_df)<=100:
            for href in l:
                url = href
                user_agent = {'User-agent': 'Mozilla/5.0'}
                result = requests.get(url, headers=user_agent)
                if result.status_code == 200:
                    print(href)
                    vacancy = result.json()
                    company_name = vacancy['employer']['name']
                    position = vacancy['name']
                    job_description = vacancy['description']
                    skills = []
                    for j in vacancy['key_skills']:
                        skills.append(j['name'])
                    key_skills = ','.join(skills)
                    ls = [company_name, position, job_description, key_skills]
                    l_df.append(ls)
                    time.sleep(1)
                else:
                    print('Ошибка обращения к серверу. Код ответа: ', result.status_code) 
        n += 1    

In [4]:
df = pd.DataFrame(data=l_df, columns=['company_name', 'position', 'job_description', 'key_skills']).astype({
                            'company_name':str,
                            'position': str,
                            'job_description': str,
                            'key_skills': str})

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     120 non-null    object
 1   position         120 non-null    object
 2   job_description  120 non-null    object
 3   key_skills       120 non-null    object
dtypes: object(4)
memory usage: 3.9+ KB


In [6]:
df

,company_name,position,job_description,key_skills
0,Webtronics,Junior/Middle backend разработчик на Fast API ...,Привет! Мы Webtronics. Ищем в команду FastAPI ...,"Git,Redis,PostgreSQL,Python,REST,REST API,Gitl..."
1,ООО ДЖАСТ ВОРК,Middle Python developer,Привет! Предлагаем опытным разработчикам присо...,"Python,Django Framework,PostgreSQL,SQL,Docker,..."
2,ООО Протектор,Middle Python (Django) Developer,Protektor – ведущий частный поставщик на рынке...,Английский — B2 — Средне-продвинутый
3,ООО Каспи Софтвер,Middle Python Developer,Обязанности: разработка серверной части (Pyth...,"Python,Django Framework,CI/CD,Docker,Backend"
4,ООО Апэрбот,Junior/Middle Python Developer (Backend) - раз...,DreamDocs.ru (ранее ApRbot) помогает компании ...,"Python,Linux,Django Framework,Bash,Docker-compose"
...,...,...,...,...
115,ООО Фитбейс,Разработчик на Python (middle),Fitbase.io - программа для учёта клиентов и ав...,"C++,Python,Удаленная работа,Алгоритмы и структ..."
116,Ростелеком - Центры обработки данных,Программист Python (Junior+/Middle),В команде разработки компании “Технический Цен...,"Git,PostgreSQL,Python,Linux,Django Framework,S..."
117,ООО AUXO (Атос АйТи Солюшенс энд Сервисез),Middle Python Developer,"Обязанности: Разработка, проверка, отладка и...","Python,PostgreSQL,Linux,Nginx,FastAPI,Django F..."
118,Mindbox,C#/Python developer junior / middle,Mindbox — крупнейшая в России облачная платфор...,"C#,ASP.NET,Python,SQL Server,TypeScript"


In [ ]:
connection = sqlite3.connect('hw1.db')

In [ ]:
df.to_sql('vacancies', 
             connection, 
             if_exists='replace',
             index=False)

In [ ]:
connection.commit()
# проверка кол-ва записей в БД
pd.read_sql("SELECT COUNT(*) FROM vacancies", connection)